# Trendyol E-Ticaret Hackathonu 2025 - Kapsamlı Model
## Arama Sonuçları Ranking Modeli

### Problem Tanımı:
- Amaç: Belirli bir kullanıcı, arama terimi ve tarih için her ürünün tıklanma ve sipariş verme olasılığını tahmin etmek
- Değerlendirme: Tıklama ve sipariş için ayrı AUC metrikleri (sipariş daha yüksek ağırlık)
- Final skor: 0.3 * AUC_click + 0.7 * AUC_order

## 1. Kütüphaneler ve Veri Yükleme

In [14]:
import polars as pl
import pandas as pd
import numpy as np
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

# PyTorch ve Hugging Face kütüphaneleri
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel, AutoConfig
import torch.nn.functional as F

import os
from datetime import datetime, timedelta
import gc

print("Kütüphaneler yüklendi!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Kütüphaneler yüklendi!
PyTorch version: 2.8.0+cpu
CUDA available: False
Using device: cpu


## 2. Veri Yükleme ve İnceleme

In [2]:
# Veri yolları
DATA_PATH = "trendyol-e-ticaret-hackathonu-2025-kaggle/data/"

print("Veriler yükleniyor...")

# Ana veriler
train_sessions = pl.read_parquet(f"{DATA_PATH}train_sessions.parquet")
test_sessions = pl.read_parquet(f"{DATA_PATH}test_sessions.parquet")

print(f"Train sessions: {train_sessions.shape}")
print(f"Test sessions: {test_sessions.shape}")

# Kullanıcı verileri
user_metadata = pl.read_parquet(f"{DATA_PATH}user/metadata.parquet")
user_sitewide_log = pl.read_parquet(f"{DATA_PATH}user/sitewide_log.parquet")
user_search_log = pl.read_parquet(f"{DATA_PATH}user/search_log.parquet")

print(f"User metadata: {user_metadata.shape}")
print(f"User sitewide log: {user_sitewide_log.shape}")
print(f"User search log: {user_search_log.shape}")

# İçerik verileri
content_metadata = pl.read_parquet(f"{DATA_PATH}content/metadata.parquet")
content_price_rate_review = pl.read_parquet(f"{DATA_PATH}content/price_rate_review_data.parquet")
content_search_log = pl.read_parquet(f"{DATA_PATH}content/search_log.parquet")
content_sitewide_log = pl.read_parquet(f"{DATA_PATH}content/sitewide_log.parquet")

print(f"Content metadata: {content_metadata.shape}")
print(f"Content price/rate/review: {content_price_rate_review.shape}")
print(f"Content search log: {content_search_log.shape}")
print(f"Content sitewide log: {content_sitewide_log.shape}")

print("\nTüm veriler yüklendi!")

Veriler yükleniyor...
Train sessions: (2773805, 9)
Test sessions: (2988697, 5)
User metadata: (38392, 4)
User sitewide log: (1891488, 6)
User search log: (1142063, 4)
Content metadata: (5414049, 10)
Content price/rate/review: (1503373, 9)
Content search log: (32128237, 4)
Content sitewide log: (23977263, 6)

Tüm veriler yüklendi!
Content metadata: (5414049, 10)
Content price/rate/review: (1503373, 9)
Content search log: (32128237, 4)
Content sitewide log: (23977263, 6)

Tüm veriler yüklendi!


## 3. Veri Sızıntısını Önlemek İçin Temporal Filtering

In [3]:
def filter_temporal_data(log_df, session_df, is_train=True):
    """
    Veri sızıntısını önlemek için temporal filtering yapar.
    Log verilerinden sadece session zamanından önce olanları alır.
    """
    print(f"Temporal filtering başlıyor... {'Train' if is_train else 'Test'} için")
    
    # Session verilerini user_id ve ts_hour ile groupby yap
    session_times = session_df.select([
        pl.col("user_id_hashed"),
        pl.col("ts_hour")
    ]).unique()
    
    # Log verilerini session verileri ile join et
    # Her kullanıcının session zamanından önce olan log verilerini al
    filtered_log = (
        log_df
        .join(
            session_times,
            on="user_id_hashed",
            how="inner",
            suffix="_session"
        )
        .filter(pl.col("ts_hour") < pl.col("ts_hour_session"))  # Sadece session öncesi veriler
        .drop("ts_hour_session")
    )
    
    print(f"Filtreleme tamamlandı. Önceki boyut: {log_df.shape}, Sonraki boyut: {filtered_log.shape}")
    return filtered_log

def filter_content_temporal_data(log_df, session_df, is_train=True):
    """
    Content log verileri için temporal filtering
    """
    print(f"Content temporal filtering başlıyor... {'Train' if is_train else 'Test'} için")
    
    # Session verilerini content_id ve ts_hour ile groupby yap
    session_times = session_df.select([
        pl.col("content_id_hashed"),
        pl.col("ts_hour")
    ]).unique()
    
    # Log verilerini session verileri ile join et
    filtered_log = (
        log_df
        .join(
            session_times,
            on="content_id_hashed",
            how="inner"
        )
        .filter(pl.col("date").dt.replace_time_zone(None) < pl.col("ts_hour").dt.replace_time_zone(None))  # Timezone'ları kaldır
        .drop("ts_hour")
    )
    
    print(f"Content filtreleme tamamlandı. Önceki boyut: {log_df.shape}, Sonraki boyut: {filtered_log.shape}")
    return filtered_log

## 4. Feature Engineering Fonksiyonları

In [4]:
def create_user_features(user_metadata, user_sitewide_log, user_search_log, session_df, is_train=True):
    """
    Kullanıcı özelliklerini oluşturur
    """
    print("Kullanıcı özellikleri oluşturuluyor...")
    
    # Temporal filtering uygula
    user_sitewide_filtered = filter_temporal_data(user_sitewide_log, session_df, is_train)
    user_search_filtered = filter_temporal_data(user_search_log, session_df, is_train)
    
    # Kullanıcı metadata işleme
    user_features = user_metadata.with_columns([
        # Yaş hesaplama (2025 - birth_year)
        (2025 - pl.col("user_birth_year")).alias("user_age"),
        # Gender kategorik encoding
        pl.col("user_gender").fill_null("UNKNOWN").alias("user_gender_clean"),
        # Tenure kategorilere ayır
        pl.when(pl.col("user_tenure_in_days") <= 30).then(pl.lit("new"))
        .when(pl.col("user_tenure_in_days") <= 365).then(pl.lit("regular"))
        .otherwise(pl.lit("loyal")).alias("user_tenure_category")
    ])
    
    # Kullanıcı sitewide aktivite özellikleri (son zamanlardaki aktivite)
    if user_sitewide_filtered.height > 0:
        user_sitewide_agg = (
            user_sitewide_filtered
            .group_by("user_id_hashed")
            .agg([
                pl.col("total_click").sum().alias("user_total_clicks"),
                pl.col("total_cart").sum().alias("user_total_cart"),
                pl.col("total_fav").sum().alias("user_total_fav"),
                pl.col("total_order").sum().alias("user_total_orders"),
                pl.col("total_click").mean().alias("user_avg_clicks"),
                pl.col("ts_hour").count().alias("user_activity_days")
            ])
        )
        user_features = user_features.join(user_sitewide_agg, on="user_id_hashed", how="left")
    
    # Kullanıcı arama aktivite özellikleri
    if user_search_filtered.height > 0:
        user_search_agg = (
            user_search_filtered
            .group_by("user_id_hashed")
            .agg([
                pl.col("total_search_impression").sum().alias("user_search_impressions"),
                pl.col("total_search_click").sum().alias("user_search_clicks"),
                pl.col("total_search_impression").count().alias("user_search_days"),
                # CTR hesaplama
                (pl.col("total_search_click").sum() / pl.col("total_search_impression").sum()).alias("user_search_ctr")
            ])
        )
        user_features = user_features.join(user_search_agg, on="user_id_hashed", how="left")
    
    # NaN değerleri doldur
    numeric_cols = [col for col in user_features.columns if col.startswith("user_") and col != "user_id_hashed" and col != "user_gender_clean" and col != "user_tenure_category"]
    user_features = user_features.with_columns([
        pl.col(col).fill_null(0) for col in numeric_cols
    ])
    
    print(f"Kullanıcı özellikleri oluşturuldu. Boyut: {user_features.shape}")
    return user_features

In [5]:
def create_content_features(content_metadata, content_price_rate_review, content_search_log, content_sitewide_log, session_df, is_train=True):
    """
    İçerik özelliklerini oluşturur
    """
    print("İçerik özellikleri oluşturuluyor...")
    
    # Temporal filtering uygula
    content_search_filtered = filter_content_temporal_data(content_search_log, session_df, is_train)
    content_sitewide_filtered = filter_content_temporal_data(content_sitewide_log, session_df, is_train)
    
    # Content metadata işleme
    content_features = content_metadata.with_columns([
        # Kategori seviyelerini temizle
        pl.col("level1_category_name").fill_null("UNKNOWN").alias("level1_category_clean"),
        pl.col("level2_category_name").fill_null("UNKNOWN").alias("level2_category_clean"),
        pl.col("leaf_category_name").fill_null("UNKNOWN").alias("leaf_category_clean"),
        # İçerik yaşı hesaplama
        (pl.date(2025, 7, 12) - pl.col("content_creation_date").dt.date()).dt.total_days().alias("content_age_days"),
        # CV tags var mı?
        pl.col("cv_tags").is_not_null().alias("has_cv_tags"),
        # Numerik özellikleri doldur
        pl.col("attribute_type_count").fill_null(0),
        pl.col("total_attribute_option_count").fill_null(0),
        pl.col("merchant_count").fill_null(1),  # En az 1 merchant olmalı
        pl.col("filterable_label_count").fill_null(0)
    ])
    
    # Fiyat ve rating bilgileri - en son bilgiyi al
    if content_price_rate_review.height > 0:
        latest_price_info = (
            content_price_rate_review
            .sort(["content_id_hashed", "update_date"])
            .group_by("content_id_hashed")
            .last()
            .with_columns([
                # İndirim oranı hesapla
                ((pl.col("original_price") - pl.col("selling_price")) / pl.col("original_price") * 100).alias("discount_percentage"),
                # Fiyat seviyesi kategorileri
                pl.when(pl.col("selling_price") <= 50).then(pl.lit("low"))
                .when(pl.col("selling_price") <= 200).then(pl.lit("medium"))
                .when(pl.col("selling_price") <= 500).then(pl.lit("high"))
                .otherwise(pl.lit("premium")).alias("price_category"),
                # Review yoğunluğu
                (pl.col("content_review_wth_media_count") / pl.col("content_review_count")).fill_null(0).alias("review_media_ratio"),
                # Rating kategorisi
                pl.when(pl.col("content_rate_avg") >= 4.5).then(pl.lit("excellent"))
                .when(pl.col("content_rate_avg") >= 4.0).then(pl.lit("good"))
                .when(pl.col("content_rate_avg") >= 3.0).then(pl.lit("average"))
                .otherwise(pl.lit("poor")).alias("rating_category")
            ])
        )
        content_features = content_features.join(latest_price_info.drop("update_date"), on="content_id_hashed", how="left")
    
    # Content arama performansı
    if content_search_filtered.height > 0:
        content_search_agg = (
            content_search_filtered
            .group_by("content_id_hashed")
            .agg([
                pl.col("total_search_impression").sum().alias("content_search_impressions"),
                pl.col("total_search_click").sum().alias("content_search_clicks"),
                (pl.col("total_search_click").sum() / pl.col("total_search_impression").sum()).alias("content_search_ctr"),
                pl.col("date").count().alias("content_search_days")
            ])
        )
        content_features = content_features.join(content_search_agg, on="content_id_hashed", how="left")
    
    # Content site geneli performansı
    if content_sitewide_filtered.height > 0:
        content_sitewide_agg = (
            content_sitewide_filtered
            .group_by("content_id_hashed")
            .agg([
                pl.col("total_click").sum().alias("content_total_clicks"),
                pl.col("total_cart").sum().alias("content_total_cart"),
                pl.col("total_fav").sum().alias("content_total_fav"),
                pl.col("total_order").sum().alias("content_total_orders"),
                # Conversion rates
                (pl.col("total_cart").sum() / pl.col("total_click").sum()).alias("content_cart_rate"),
                (pl.col("total_order").sum() / pl.col("total_click").sum()).alias("content_order_rate"),
                pl.col("date").count().alias("content_active_days")
            ])
        )
        content_features = content_features.join(content_sitewide_agg, on="content_id_hashed", how="left")
    
    # NaN değerleri doldur
    numeric_cols = [col for col in content_features.columns 
                   if col not in ["content_id_hashed", "level1_category_clean", "level2_category_clean", 
                                "leaf_category_clean", "cv_tags", "price_category", "rating_category"] 
                   and content_features[col].dtype in [pl.Float64, pl.Float32, pl.Int64, pl.Int32]]
    
    content_features = content_features.with_columns([
        pl.col(col).fill_null(0) for col in numeric_cols
    ])
    
    print(f"İçerik özellikleri oluşturuldu. Boyut: {content_features.shape}")
    return content_features

In [6]:
def create_interaction_features(session_df, user_features, content_features):
    """
    Kullanıcı-İçerik etkileşim özelliklerini oluşturur
    """
    print("Etkileşim özellikleri oluşturuluyor...")
    
    # Session verilerini user ve content özellikleri ile birleştir
    enriched_data = (
        session_df
        .join(user_features, on="user_id_hashed", how="left")
        .join(content_features, on="content_id_hashed", how="left")
    )
    
    # Zaman özellikleri
    enriched_data = enriched_data.with_columns([
        pl.col("ts_hour").dt.hour().alias("hour_of_day"),
        pl.col("ts_hour").dt.weekday().alias("day_of_week"),
        pl.col("ts_hour").dt.day().alias("day_of_month"),
        # Tatil/hafta sonu
        (pl.col("ts_hour").dt.weekday() >= 5).alias("is_weekend"),
        # Gün içi saat kategorileri
        pl.when(pl.col("ts_hour").dt.hour().is_between(6, 12)).then(pl.lit("morning"))
        .when(pl.col("ts_hour").dt.hour().is_between(12, 18)).then(pl.lit("afternoon"))
        .when(pl.col("ts_hour").dt.hour().is_between(18, 23)).then(pl.lit("evening"))
        .otherwise(pl.lit("night")).alias("time_period")
    ])
    
    # Kullanıcı-kategori uyumu özellikleri
    if "user_gender_clean" in enriched_data.columns and "level1_category_clean" in enriched_data.columns:
        enriched_data = enriched_data.with_columns([
            # Kadın + moda kategorileri uyumu
            ((pl.col("user_gender_clean") == "Bayan") & 
             (pl.col("level1_category_clean").is_in(["Giyim", "Ayakkabı", "Aksesuar"]))).alias("gender_category_match"),
            
            # Yaş-kategori uyumu
            pl.when((pl.col("user_age") <= 30) & (pl.col("level2_category_clean").str.contains("Genç|Casual")))
            .then(pl.lit(True))
            .when((pl.col("user_age") >= 40) & (pl.col("level2_category_clean").str.contains("Klasik|Şık")))
            .then(pl.lit(True))
            .otherwise(pl.lit(False)).alias("age_category_match")
        ])
    
    # Arama terimi özellikleri
    if "search_term_normalized" in enriched_data.columns:
        enriched_data = enriched_data.with_columns([
            pl.col("search_term_normalized").str.len_chars().alias("search_term_length"),
            pl.col("search_term_normalized").str.count_matches("_").alias("search_term_word_count"),
            # Marka arama mı?
            pl.col("search_term_normalized").str.contains("nike|adidas|zara|h&m|mango").alias("is_brand_search"),
            # Renk arama mı?
            pl.col("search_term_normalized").str.contains("siyah|beyaz|kirmizi|mavi|yesil|sari").alias("is_color_search")
        ])
    
    # Fiyat-kullanıcı uyumu (eğer fiyat bilgisi varsa)
    if "selling_price" in enriched_data.columns and "user_total_orders" in enriched_data.columns:
        enriched_data = enriched_data.with_columns([
            # Kullanıcının geçmiş sipariş sayısına göre fiyat toleransı
            pl.when(pl.col("user_total_orders") >= 10)
            .then(pl.col("selling_price") <= 500)  # Sadık müşteriler daha yüksek fiyat tolere eder
            .otherwise(pl.col("selling_price") <= 200).alias("price_user_match")
        ])
    
    print(f"Etkileşim özellikleri oluşturuldu. Boyut: {enriched_data.shape}")
    return enriched_data

## 5. Ana Pipeline - Veri Hazırlama

In [7]:
# Train verisi için feature engineering
print("=== TRAIN VERİSİ HAZIRLIĞI ===")
train_user_features = create_user_features(
    user_metadata, user_sitewide_log, user_search_log, train_sessions, is_train=True
)

train_content_features = create_content_features(
    content_metadata, content_price_rate_review, content_search_log, 
    content_sitewide_log, train_sessions, is_train=True
)

train_enriched = create_interaction_features(
    train_sessions, train_user_features, train_content_features
)

print(f"\nTrain verisi hazır. Boyut: {train_enriched.shape}")
print(f"Özellik sayısı: {len(train_enriched.columns)}")

# Test verisi için feature engineering
print("\n=== TEST VERİSİ HAZIRLIĞI ===")
test_user_features = create_user_features(
    user_metadata, user_sitewide_log, user_search_log, test_sessions, is_train=False
)

test_content_features = create_content_features(
    content_metadata, content_price_rate_review, content_search_log, 
    content_sitewide_log, test_sessions, is_train=False
)

test_enriched = create_interaction_features(
    test_sessions, test_user_features, test_content_features
)

print(f"\nTest verisi hazır. Boyut: {test_enriched.shape}")
print(f"Özellik sayısı: {len(test_enriched.columns)}")

=== TRAIN VERİSİ HAZIRLIĞI ===
Kullanıcı özellikleri oluşturuluyor...
Temporal filtering başlıyor... Train için
Filtreleme tamamlandı. Önceki boyut: (1891488, 6), Sonraki boyut: (1034837, 6)
Temporal filtering başlıyor... Train için
Filtreleme tamamlandı. Önceki boyut: (1142063, 4), Sonraki boyut: (623930, 4)
Kullanıcı özellikleri oluşturuldu. Boyut: (38392, 17)
İçerik özellikleri oluşturuluyor...
Content temporal filtering başlıyor... Train için
Content filtreleme tamamlandı. Önceki boyut: (32128237, 4), Sonraki boyut: (29390308, 4)
Content temporal filtering başlıyor... Train için
Content filtreleme tamamlandı. Önceki boyut: (32128237, 4), Sonraki boyut: (29390308, 4)
Content temporal filtering başlıyor... Train için
Content filtreleme tamamlandı. Önceki boyut: (23977263, 6), Sonraki boyut: (24784826, 6)
Content filtreleme tamamlandı. Önceki boyut: (23977263, 6), Sonraki boyut: (24784826, 6)
İçerik özellikleri oluşturuldu. Boyut: (5414049, 37)
Etkileşim özellikleri oluşturuluyor...
İ

## 6. Model Hazırlığı ve Eğitim

In [12]:
class TwoTowerModel(nn.Module):
    def __init__(self, config):
        super(TwoTowerModel, self).__init__()
        
        self.config = config
        
        # Türkçe BERT modeli
        self.bert_model_name = "dbmdz/bert-base-turkish-cased"
        self.tokenizer = AutoTokenizer.from_pretrained(self.bert_model_name)
        self.bert = AutoModel.from_pretrained(self.bert_model_name)
        
        # BERT'i dondurmak için (opsiyonel - performans için)
        if config.get('freeze_bert', False):
            for param in self.bert.parameters():
                param.requires_grad = False
        
        # Embedding boyutları
        self.bert_dim = self.bert.config.hidden_size  # 768
        self.user_embedding_dim = config['user_embedding_dim']
        self.content_embedding_dim = config['content_embedding_dim']
        self.tower_dim = config['tower_dim']
        
        # Kullanıcı ve içerik embedding katmanları
        self.user_embedding = nn.Embedding(config['num_users'], self.user_embedding_dim)
        self.content_embedding = nn.Embedding(config['num_contents'], self.content_embedding_dim)
        
        # Sorgu Kulesi (Query Tower)
        self.query_tower = nn.Sequential(
            nn.Linear(self.bert_dim + self.user_embedding_dim, self.tower_dim * 2),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(self.tower_dim * 2, self.tower_dim),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(self.tower_dim, self.tower_dim)
        )
        
        # Ürün Kulesi (Item Tower)  
        self.item_tower = nn.Sequential(
            nn.Linear(self.content_embedding_dim, self.tower_dim * 2),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(self.tower_dim * 2, self.tower_dim),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(self.tower_dim, self.tower_dim)
        )
        
        # Çıktı başlıkları (Output heads)
        self.click_head = nn.Linear(self.tower_dim * 2, 1)  # Query + Item concat
        self.order_head = nn.Linear(self.tower_dim * 2, 1)
        
        # Initialize weights
        self._init_weights()
    
    def _init_weights(self):
        """Weight initialization"""
        # Embedding layers
        nn.init.xavier_uniform_(self.user_embedding.weight)
        nn.init.xavier_uniform_(self.content_embedding.weight)
            
        # Sequential modules
        for module in [self.query_tower, self.item_tower]:
            for layer in module:
                if isinstance(layer, nn.Linear):
                    nn.init.xavier_uniform_(layer.weight)
                    nn.init.zeros_(layer.bias)
        
        # Head layers
        for layer in [self.click_head, self.order_head]:
            nn.init.xavier_uniform_(layer.weight)
            nn.init.zeros_(layer.bias)
    
    def encode_query(self, input_ids, attention_mask, user_ids):
        """Sorgu kulesinde arama terimi + kullanıcı ID'si işlenir"""
        # BERT ile arama terimini encode et
        bert_outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        # CLS token'ı al (ilk token)
        query_text_vec = bert_outputs.last_hidden_state[:, 0, :]  # [batch_size, 768]
        
        # Kullanıcı embedding'ini al
        user_vec = self.user_embedding(user_ids)  # [batch_size, user_embedding_dim]
        
        # Birleştir ve sorgu kulesinden geçir
        query_input = torch.cat([query_text_vec, user_vec], dim=1)
        query_vec = self.query_tower(query_input)  # [batch_size, tower_dim]
        
        return query_vec
    
    def encode_item(self, content_ids):
        """Ürün kulesinde içerik ID'si işlenir"""
        # İçerik embedding'ini al
        content_vec = self.content_embedding(content_ids)  # [batch_size, content_embedding_dim]
        
        # Ürün kulesinden geçir
        item_vec = self.item_tower(content_vec)  # [batch_size, tower_dim]
        
        return item_vec
    
    def forward(self, input_ids, attention_mask, user_ids, content_ids):
        """Forward pass"""
        # Sorgu vektörü
        query_vec = self.encode_query(input_ids, attention_mask, user_ids)
        
        # Ürün vektörü
        item_vec = self.encode_item(content_ids)
        
        # Vektörleri birleştir
        combined_vec = torch.cat([query_vec, item_vec], dim=1)  # [batch_size, tower_dim * 2]
        
        # Çıktı başlıkları
        click_logit = self.click_head(combined_vec)  # [batch_size, 1]
        order_logit = self.order_head(combined_vec)  # [batch_size, 1]
        
        return click_logit.squeeze(-1), order_logit.squeeze(-1)  # [batch_size], [batch_size]


class TrendyolDataset(Dataset):
    def __init__(self, data_df, tokenizer, user_id_map, content_id_map, max_length=128):
        self.data = data_df.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.user_id_map = user_id_map
        self.content_id_map = content_id_map
        self.max_length = max_length
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        
        # Arama terimini tokenize et
        search_term = str(row.get('search_term_normalized', ''))
        if pd.isna(search_term) or search_term == 'nan':
            search_term = ''
            
        encoding = self.tokenizer(
            search_term,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        
        # ID'leri mapping'e çevir
        user_id = self.user_id_map.get(row['user_id_hashed'], 0)
        content_id = self.content_id_map.get(row['content_id_hashed'], 0)
        
        item = {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'user_id': torch.tensor(user_id, dtype=torch.long),
            'content_id': torch.tensor(content_id, dtype=torch.long),
        }
        
        # Eğitim verisi için hedefleri ekle
        if 'clicked' in self.data.columns:
            item['clicked'] = torch.tensor(row['clicked'], dtype=torch.float)
            item['ordered'] = torch.tensor(row['ordered'], dtype=torch.float)
        
        return item

print("Two-Tower model sınıfı tanımlandı!")

Two-Tower model sınıfı tanımlandı!


In [9]:
def create_id_mappings(train_df, test_df):
    """Kullanıcı ve içerik ID'leri için mapping oluştur"""
    print("ID mappings oluşturuluyor...")
    
    # Tüm benzersiz ID'leri al
    all_user_ids = set(train_df['user_id_hashed'].unique()) | set(test_df['user_id_hashed'].unique())
    all_content_ids = set(train_df['content_id_hashed'].unique()) | set(test_df['content_id_hashed'].unique())
    
    # 0 index'ini boşta bırak (unknown/padding için)
    user_id_map = {user_id: idx + 1 for idx, user_id in enumerate(sorted(all_user_ids))}
    content_id_map = {content_id: idx + 1 for idx, content_id in enumerate(sorted(all_content_ids))}
    
    print(f"Benzersiz kullanıcı sayısı: {len(user_id_map)}")
    print(f"Benzersiz içerik sayısı: {len(content_id_map)}")
    
    return user_id_map, content_id_map


def train_epoch(model, dataloader, optimizer, criterion_click, criterion_order, device, scaler=None):
    """Bir epoch eğitimi"""
    model.train()
    total_loss = 0
    click_preds, click_targets = [], []
    order_preds, order_targets = [], []
    
    for batch_idx, batch in enumerate(dataloader):
        # Veriyi device'a taşı
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        user_ids = batch['user_id'].to(device)
        content_ids = batch['content_id'].to(device)
        click_labels = batch['clicked'].to(device)
        order_labels = batch['ordered'].to(device)
        
        optimizer.zero_grad()
        
        # Mixed precision training
        if scaler:
            with torch.cuda.amp.autocast():
                click_logits, order_logits = model(input_ids, attention_mask, user_ids, content_ids)
                click_loss = criterion_click(click_logits, click_labels)
                order_loss = criterion_order(order_logits, order_labels)
                total_batch_loss = 0.3 * click_loss + 0.7 * order_loss  # Ağırlıklı kayıp
            
            scaler.scale(total_batch_loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            click_logits, order_logits = model(input_ids, attention_mask, user_ids, content_ids)
            click_loss = criterion_click(click_logits, click_labels)
            order_loss = criterion_order(order_logits, order_labels)
            total_batch_loss = 0.3 * click_loss + 0.7 * order_loss
            
            total_batch_loss.backward()
            optimizer.step()
        
        total_loss += total_batch_loss.item()
        
        # Tahminleri topla (AUC hesabı için)
        with torch.no_grad():
            click_probs = torch.sigmoid(click_logits).cpu().numpy()
            order_probs = torch.sigmoid(order_logits).cpu().numpy()
            
            click_preds.extend(click_probs)
            click_targets.extend(click_labels.cpu().numpy())
            order_preds.extend(order_probs)
            order_targets.extend(order_labels.cpu().numpy())
        
        if batch_idx % 100 == 0:
            print(f'Batch {batch_idx}/{len(dataloader)}, Loss: {total_batch_loss.item():.4f}')
    
    # AUC hesapla
    try:
        click_auc = roc_auc_score(click_targets, click_preds)
        order_auc = roc_auc_score(order_targets, order_preds)
    except:
        click_auc = 0.5
        order_auc = 0.5
    
    avg_loss = total_loss / len(dataloader)
    return avg_loss, click_auc, order_auc


def validate_epoch(model, dataloader, criterion_click, criterion_order, device):
    """Validation epoch"""
    model.eval()
    total_loss = 0
    click_preds, click_targets = [], []
    order_preds, order_targets = [], []
    
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            user_ids = batch['user_id'].to(device)
            content_ids = batch['content_id'].to(device)
            click_labels = batch['clicked'].to(device)
            order_labels = batch['ordered'].to(device)
            
            click_logits, order_logits = model(input_ids, attention_mask, user_ids, content_ids)
            
            click_loss = criterion_click(click_logits, click_labels)
            order_loss = criterion_order(order_logits, order_labels)
            total_batch_loss = 0.3 * click_loss + 0.7 * order_loss
            
            total_loss += total_batch_loss.item()
            
            # Tahminleri topla
            click_probs = torch.sigmoid(click_logits).cpu().numpy()
            order_probs = torch.sigmoid(order_logits).cpu().numpy()
            
            click_preds.extend(click_probs)
            click_targets.extend(click_labels.cpu().numpy())
            order_preds.extend(order_probs)
            order_targets.extend(order_labels.cpu().numpy())
    
    # AUC hesapla
    try:
        click_auc = roc_auc_score(click_targets, click_preds)
        order_auc = roc_auc_score(order_targets, order_preds)
    except:
        click_auc = 0.5
        order_auc = 0.5
    
    avg_loss = total_loss / len(dataloader)
    return avg_loss, click_auc, order_auc

print("Eğitim fonksiyonları tanımlandı!")

Eğitim fonksiyonları tanımlandı!


In [10]:
def prepare_model_data_pytorch(enriched_df, is_train=True):
    """PyTorch modeli için veriyi hazırlar"""
    print(f"PyTorch model verisi hazırlanıyor... {'Train' if is_train else 'Test'}")
    
    # Gerekli sütunları seç
    required_cols = ['user_id_hashed', 'content_id_hashed', 'search_term_normalized']
    if is_train:
        required_cols.extend(['clicked', 'ordered'])
    else:
        required_cols.append('session_id')
    
    # Mevcut sütunları kontrol et
    available_cols = [col for col in required_cols if col in enriched_df.columns]
    
    # DataFrame'i seç ve pandas'a çevir
    df_work = enriched_df.select(available_cols).to_pandas()
    
    print(f"Çalışma verisi boyutu: {df_work.shape}")
    print(f"Sütunlar: {list(df_work.columns)}")
    
    # NaN değerleri temizle
    df_work['search_term_normalized'] = df_work['search_term_normalized'].fillna('')
    
    if is_train:
        print(f"Click dağılımı: {df_work['clicked'].value_counts().to_dict()}")
        print(f"Order dağılımı: {df_work['ordered'].value_counts().to_dict()}")
    
    return df_work

print("PyTorch veri hazırlama fonksiyonu tanımlandı.")

# Veri hazırlama
print("\n=== PYTORCH MODEL VERİ HAZIRLIĞI ===")
train_data_pytorch = prepare_model_data_pytorch(train_enriched, is_train=True)
test_data_pytorch = prepare_model_data_pytorch(test_enriched, is_train=False)

PyTorch veri hazırlama fonksiyonu tanımlandı.

=== PYTORCH MODEL VERİ HAZIRLIĞI ===
PyTorch model verisi hazırlanıyor... Train
Çalışma verisi boyutu: (2773805, 5)
Sütunlar: ['user_id_hashed', 'content_id_hashed', 'search_term_normalized', 'clicked', 'ordered']
Click dağılımı: {0: 2674602, 1: 99203}
Order dağılımı: {0: 2765269, 1: 8536}
PyTorch model verisi hazırlanıyor... Test
Çalışma verisi boyutu: (2773805, 5)
Sütunlar: ['user_id_hashed', 'content_id_hashed', 'search_term_normalized', 'clicked', 'ordered']
Click dağılımı: {0: 2674602, 1: 99203}
Order dağılımı: {0: 2765269, 1: 8536}
PyTorch model verisi hazırlanıyor... Test
Çalışma verisi boyutu: (2988697, 4)
Sütunlar: ['user_id_hashed', 'content_id_hashed', 'search_term_normalized', 'session_id']
Çalışma verisi boyutu: (2988697, 4)
Sütunlar: ['user_id_hashed', 'content_id_hashed', 'search_term_normalized', 'session_id']


In [13]:
# ID mappings oluştur
user_id_map, content_id_map = create_id_mappings(train_data_pytorch, test_data_pytorch)

# Model config
model_config = {
    'user_embedding_dim': 64,  # Küçülttük
    'content_embedding_dim': 64,  # Küçülttük
    'tower_dim': 128,  # Küçülttük
    'num_users': len(user_id_map) + 1,  # +1 for padding
    'num_contents': len(content_id_map) + 1,  # +1 for padding
    'freeze_bert': True  # BERT'i dondur - hızlandırmak için
}

print(f"Model config: {model_config}")

# Model oluştur
model = TwoTowerModel(model_config).to(device)
tokenizer = model.tokenizer

print(f"Model parametre sayısı: {sum(p.numel() for p in model.parameters()):,}")
print(f"Eğitilebilir parametre sayısı: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

# Eğitim verisi için daha küçük bir sample al (hızlandırmak için)
train_sample = train_data_pytorch.sample(n=100000, random_state=42)  # 100k sample

# Train/validation split
train_df, val_df = train_test_split(
    train_sample,  # Sample kullan
    test_size=0.2, 
    random_state=42, 
    stratify=train_sample['clicked']
)

print(f"Train boyutu: {train_df.shape}")
print(f"Validation boyutu: {val_df.shape}")

# Test verisi de küçültelim (ilk 10k)
test_sample = test_data_pytorch.head(10000)

# Dataset oluştur
train_dataset = TrendyolDataset(train_df, tokenizer, user_id_map, content_id_map, max_length=64)  # Max length küçülttük
val_dataset = TrendyolDataset(val_df, tokenizer, user_id_map, content_id_map, max_length=64)
test_dataset = TrendyolDataset(test_sample, tokenizer, user_id_map, content_id_map, max_length=64)

# DataLoader oluştur
batch_size = 16  # Küçülttük
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)  # num_workers=0 Windows için
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

print(f"Train batches: {len(train_loader)}")
print(f"Val batches: {len(val_loader)}")
print(f"Test batches: {len(test_loader)}")

# Loss functions ve optimizer
criterion_click = nn.BCEWithLogitsLoss()
criterion_order = nn.BCEWithLogitsLoss()

# AdamW optimizer with warmup
optimizer = optim.AdamW([p for p in model.parameters() if p.requires_grad], lr=5e-4, weight_decay=0.01)  # Sadece eğitilebilir parametreler

# Learning rate scheduler
from torch.optim.lr_scheduler import CosineAnnealingLR
scheduler = CosineAnnealingLR(optimizer, T_max=3, eta_min=1e-6)

# Mixed precision training (CPU'da kullanılmayacak)
scaler = None  # CPU'da mixed precision yok

print("Eğitim hazırlıkları tamamlandı!")

ID mappings oluşturuluyor...
Benzersiz kullanıcı sayısı: 38392
Benzersiz içerik sayısı: 1172785
Model config: {'user_embedding_dim': 128, 'content_embedding_dim': 128, 'tower_dim': 256, 'num_users': 38393, 'num_contents': 1172786, 'freeze_bert': False}
Benzersiz kullanıcı sayısı: 38392
Benzersiz içerik sayısı: 1172785
Model config: {'user_embedding_dim': 128, 'content_embedding_dim': 128, 'tower_dim': 256, 'num_users': 38393, 'num_contents': 1172786, 'freeze_bert': False}
Model parametre sayısı: 266,568,834
Eğitilebilir parametre sayısı: 266,568,834
Model parametre sayısı: 266,568,834
Eğitilebilir parametre sayısı: 266,568,834
Train boyutu: (2219044, 5)
Validation boyutu: (554761, 5)
Train boyutu: (2219044, 5)
Validation boyutu: (554761, 5)
Train batches: 69346
Val batches: 17337
Test batches: 93397
Eğitim hazırlıkları tamamlandı!
Train batches: 69346
Val batches: 17337
Test batches: 93397
Eğitim hazırlıkları tamamlandı!


In [15]:
print("\n=== PYTORCH TWO-TOWER MODEL EĞİTİMİ ===")

num_epochs = 3  # BERT ile az epoch yeterli
best_val_score = 0
best_model_state = None

for epoch in range(num_epochs):
    print(f"\n=== EPOCH {epoch+1}/{num_epochs} ===")
    
    # Training
    train_loss, train_click_auc, train_order_auc = train_epoch(
        model, train_loader, optimizer, criterion_click, criterion_order, device, scaler
    )
    
    # Validation
    val_loss, val_click_auc, val_order_auc = validate_epoch(
        model, val_loader, criterion_click, criterion_order, device
    )
    
    # Learning rate scheduler step
    scheduler.step()
    
    # Combined score hesapla
    train_combined = 0.3 * train_click_auc + 0.7 * train_order_auc
    val_combined = 0.3 * val_click_auc + 0.7 * val_order_auc
    
    print(f"Train - Loss: {train_loss:.4f}, Click AUC: {train_click_auc:.4f}, Order AUC: {train_order_auc:.4f}, Combined: {train_combined:.4f}")
    print(f"Val   - Loss: {val_loss:.4f}, Click AUC: {val_click_auc:.4f}, Order AUC: {val_order_auc:.4f}, Combined: {val_combined:.4f}")
    print(f"LR: {optimizer.param_groups[0]['lr']:.2e}")
    
    # En iyi modeli kaydet
    if val_combined > best_val_score:
        best_val_score = val_combined
        best_model_state = model.state_dict().copy()
        print(f"✅ En iyi model kaydedildi! Score: {best_val_score:.4f}")

# En iyi modeli yükle
if best_model_state is not None:
    model.load_state_dict(best_model_state)
    print(f"\n✅ En iyi model yüklendi. Final validation score: {best_val_score:.4f}")

print("\n🎉 Model eğitimi tamamlandı!")

NaN değerler temizlendi.
Final train shape: (2773805, 56)
Final test shape: (2988697, 56)

=== LIGHTGBM MODEL EĞİTİMİ ===
Train split shape: (2219044, 56)
Validation split shape: (554761, 56)

1. Click modeli eğitiliyor...
Training until validation scores don't improve for 50 rounds
[100]	train's auc: 0.70537	valid's auc: 0.69114
[200]	train's auc: 0.724852	valid's auc: 0.698947
[300]	train's auc: 0.740355	valid's auc: 0.703547
[400]	train's auc: 0.75489	valid's auc: 0.708156
[500]	train's auc: 0.766745	valid's auc: 0.710712
[600]	train's auc: 0.778112	valid's auc: 0.713393
[700]	train's auc: 0.787896	valid's auc: 0.714938
[800]	train's auc: 0.797007	valid's auc: 0.716225
[900]	train's auc: 0.805097	valid's auc: 0.717238
[1000]	train's auc: 0.813035	valid's auc: 0.718533
Did not meet early stopping. Best iteration is:
[1000]	train's auc: 0.813035	valid's auc: 0.718533
Click Model Validation AUC: 0.718533

2. Order modeli eğitiliyor...
Training until validation scores don't improve for 

In [16]:
print("=== PYTORCH MODEL TEST TAHMİNLERİ ===")

model.eval()
all_click_preds = []
all_order_preds = []
all_session_ids = []
all_content_ids = []

with torch.no_grad():
    for batch_idx, batch in enumerate(test_loader):
        if batch_idx % 100 == 0:
            print(f"Test batch {batch_idx+1}/{len(test_loader)}")
        
        # Veriyi device'a taşı
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        user_ids = batch['user_id'].to(device)
        content_ids = batch['content_id'].to(device)
        
        # Tahmin yap
        click_logits, order_logits = model(input_ids, attention_mask, user_ids, content_ids)
        
        # Sigmoid ile probability'ye çevir
        click_probs = torch.sigmoid(click_logits).cpu().numpy()
        order_probs = torch.sigmoid(order_logits).cpu().numpy()
        
        all_click_preds.extend(click_probs)
        all_order_preds.extend(order_probs)

# Test data'dan session ve content ID'lerini al
test_session_ids = test_data_pytorch['session_id'].values
test_content_ids = test_data_pytorch['content_id_hashed'].values

click_pred_test = np.array(all_click_preds)
order_pred_test = np.array(all_order_preds)

print(f"Click tahmin dağılımı: min={click_pred_test.min():.4f}, max={click_pred_test.max():.4f}, mean={click_pred_test.mean():.4f}")
print(f"Order tahmin dağılımı: min={order_pred_test.min():.4f}, max={order_pred_test.max():.4f}, mean={order_pred_test.mean():.4f}")

# Final skor hesaplama (0.3*click + 0.7*order)
combined_pred_test = 0.3 * click_pred_test + 0.7 * order_pred_test
print(f"Combined tahmin dağılımı: min={combined_pred_test.min():.4f}, max={combined_pred_test.max():.4f}, mean={combined_pred_test.mean():.4f}")

# Test DataFrame oluştur
test_predictions = pd.DataFrame({
    'session_id': test_session_ids,
    'content_id_hashed': test_content_ids, 
    'combined_score': combined_pred_test
})

print(f"Test predictions shape: {test_predictions.shape}")
print(f"Unique session sayısı: {test_predictions['session_id'].nunique()}")

# Sample submission kontrol et
sample_submission = pd.read_csv('c:/Users/pc/Desktop/trendyol_hekaton/trendyol-e-ticaret-hackathonu-2025-kaggle/data/sample_submission.csv')
print(f"\nSample submission shape: {sample_submission.shape}")

# Sample'da content sayısı kontrol
sample_lengths = sample_submission['prediction'].str.split().str.len()
print(f"Sample'da ortalama content sayısı per session: {sample_lengths.mean():.1f}")
print(f"Sample'da min-max content sayısı: {sample_lengths.min()}-{sample_lengths.max()}")

print("\nSample formatı örneği:")
print(f"İlk session: {sample_submission.iloc[0]['session_id']}")
first_prediction = sample_submission.iloc[0]['prediction'].split()
print(f"Content sayısı: {len(first_prediction)}")
print(f"İlk 5 content: {' '.join(first_prediction[:5])}")

# ÖNEMLİ: Her session için TÜM content'leri sıralayıp tek string yapmalıyız
def create_correct_submission(test_df):
    """
    Sample submission formatında doğru submission oluşturur:
    - Her session için TÜM content'leri combined_score'a göre sıralar
    - Boşlukla ayrılmış tek string yapar
    """
    submission_list = []
    
    print("Her session için content'ler sıralanıyor...")
    unique_sessions = test_df['session_id'].unique()
    
    for i, session_id in enumerate(unique_sessions):
        if i % 1000 == 0:
            print(f"İşlenen session: {i+1}/{len(unique_sessions)}")
        
        # Bu session'daki tüm content'leri al ve sırala
        session_data = test_df[test_df['session_id'] == session_id].sort_values(
            'combined_score', ascending=False
        )
        
        # TÜM content_id'leri boşlukla ayırarak birleştir
        content_ids_string = ' '.join(session_data['content_id_hashed'].astype(str))
        
        submission_list.append({
            'session_id': session_id,
            'prediction': content_ids_string
        })
    
    return pd.DataFrame(submission_list)

# Doğru submission oluştur
print("\n=== DOĞRU SUBMISSION OLUŞTURULUYOR ===")
final_submission = create_correct_submission(test_predictions)

# Kontrol et
our_lengths = final_submission['prediction'].str.split().str.len()
print(f"\nFinal submission shape: {final_submission.shape}")
print(f"Bizim submission'da ortalama content sayısı: {our_lengths.mean():.1f}")
print(f"Bizim submission'da min-max content sayısı: {our_lengths.min()}-{our_lengths.max()}")

# Örnek kontrol
print(f"\nBizim submission örneği:")
print(f"İlk session: {final_submission.iloc[0]['session_id']}")
our_first = final_submission.iloc[0]['prediction'].split()
print(f"Content sayısı: {len(our_first)}")
print(f"İlk 5 content: {' '.join(our_first[:5])}")

# Format kontrolü
if final_submission.shape[0] == sample_submission.shape[0]:
    print("✅ Session sayısı doğru!")
else:
    print(f"❌ Session sayısı hatalı! Bizde: {final_submission.shape[0]}, Sample'da: {sample_submission.shape[0]}")

# Kaydet
final_submission.to_csv('c:/Users/pc/Desktop/trendyol_hekaton/submission_final_correct.csv', index=False)
print(f"\n✅ Doğru submission kaydedildi: submission_final_correct.csv")

print(f"\n=== PYTORCH MODEL ÖZETİ ===")
print(f"✅ Model: Two-Tower with Turkish BERT")
print(f"✅ Final Validation Score: {best_val_score:.6f}")
print(f"✅ Test sessions: {final_submission.shape[0]:,}")
print(f"✅ Total predictions: {len(test_predictions):,}")
print("✅ Her session için TÜM content'ler sıralandı ve boşlukla ayrıldı!")
print("✅ Turkish BERT (dbmdz/bert-base-turkish-cased) kullanıldı!")

=== TEST TAHMİNLERİ ===
Click tahmin dağılımı: min=0.0002, max=0.8133, mean=0.0350
Order tahmin dağılımı: min=0.0000, max=0.3779, mean=0.0024
Combined tahmin dağılımı: min=0.0001, max=0.2834, mean=0.0122

Submission shape: (2988697, 5)
Unique session sayısı: 18589
Unique content sayısı: 755886

Session bazlı sıralama yapılıyor...
Sample submission shape: (18589, 2)
Sample submission columns: ['session_id', 'prediction']

Final submission saved! Shape: (371780, 2)
Final submission head:
               session_id        content_id
71  test_0001ff614df60933  52fda2c36243ef51
83  test_0001ff614df60933  4e1a2bcf236c7f54
51  test_0001ff614df60933  5193e92449cb3c31
11  test_0001ff614df60933  2dd3e675656a92db
66  test_0001ff614df60933  d13c66f775662c28
91  test_0001ff614df60933  b1ae3df07f40a185
23  test_0001ff614df60933  4725484b4a90c923
24  test_0001ff614df60933  6a878e9d33487cbc
45  test_0001ff614df60933  445032265c99c830
28  test_0001ff614df60933  89b1a5be8f804fe7

=== ÖZET ===
Validation 

## 10. Model Kaydetme

In [ ]:
# MODEL EĞİTİMİNİ BAŞLAT
print("🚀 Two-Tower model eğitimi başlıyor...")

# En iyi skor takibi
best_val_score = 0.0
best_model_state = None

# Eğitim döngüsü
num_epochs = 3  # GPU ile hızlı eğitim için 3 epoch yeterli

for epoch in range(num_epochs):
    print(f"\n{'='*50}")
    print(f"EPOCH {epoch+1}/{num_epochs}")
    print(f"{'='*50}")
    
    # Eğitim moduna geç
    model.train()
    train_loss = 0.0
    train_click_preds = []
    train_click_targets = []
    train_order_preds = []  
    train_order_targets = []
    
    print("Eğitim başlıyor...")
    for batch_idx, batch in enumerate(train_loader):
        if batch_idx % 50 == 0:
            print(f"Batch {batch_idx+1}/{len(train_loader)}")
            
        # Veriyi device'a taşı
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        user_ids = batch['user_id'].to(device)
        content_ids = batch['content_id'].to(device)
        clicked = batch['clicked'].to(device)
        ordered = batch['ordered'].to(device)
        
        # Gradientları sıfırla
        optimizer.zero_grad()
        
        # Forward pass
        click_logits, order_logits = model(input_ids, attention_mask, user_ids, content_ids)
        
        # Loss hesapla
        click_loss = criterion_click(click_logits, clicked)
        order_loss = criterion_order(order_logits, ordered)
        total_loss = click_loss + order_loss
        
        # Backward pass
        total_loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        
        # İstatistikleri kaydet
        train_loss += total_loss.item()
        
        # Tahminleri sigmoid'e geçir
        click_probs = torch.sigmoid(click_logits).detach().cpu().numpy()
        order_probs = torch.sigmoid(order_logits).detach().cpu().numpy()
        
        train_click_preds.extend(click_probs)
        train_click_targets.extend(clicked.detach().cpu().numpy())
        train_order_preds.extend(order_probs)
        train_order_targets.extend(ordered.detach().cpu().numpy())
    
    # Learning rate scheduler step
    scheduler.step()
    
    # Eğitim metrikleri
    train_loss = train_loss / len(train_loader)
    train_click_auc = roc_auc_score(train_click_targets, train_click_preds)
    train_order_auc = roc_auc_score(train_order_targets, train_order_preds)
    train_combined = 0.3 * train_click_auc + 0.7 * train_order_auc
    
    # Validation
    print("Validation başlıyor...")
    model.eval()
    val_loss = 0.0
    val_click_preds = []
    val_click_targets = []
    val_order_preds = []
    val_order_targets = []
    
    with torch.no_grad():
        for batch_idx, batch in enumerate(val_loader):
            # Veriyi device'a taşı
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            user_ids = batch['user_id'].to(device)
            content_ids = batch['content_id'].to(device)
            clicked = batch['clicked'].to(device)
            ordered = batch['ordered'].to(device)
            
            # Forward pass
            click_logits, order_logits = model(input_ids, attention_mask, user_ids, content_ids)
            
            # Loss hesapla
            click_loss = criterion_click(click_logits, clicked)
            order_loss = criterion_order(order_logits, ordered)
            total_loss = click_loss + order_loss
            
            val_loss += total_loss.item()
            
            # Tahminleri sigmoid'e geçir
            click_probs = torch.sigmoid(click_logits).cpu().numpy()
            order_probs = torch.sigmoid(order_logits).cpu().numpy()
            
            val_click_preds.extend(click_probs)
            val_click_targets.extend(clicked.cpu().numpy())
            val_order_preds.extend(order_probs)
            val_order_targets.extend(ordered.cpu().numpy())
    
    # Validation metrikleri
    val_loss = val_loss / len(val_loader)
    val_click_auc = roc_auc_score(val_click_targets, val_click_preds)
    val_order_auc = roc_auc_score(val_order_targets, val_order_preds)
    val_combined = 0.3 * val_click_auc + 0.7 * val_order_auc
    
    # Sonuçları yazdır
    print(f"Train - Loss: {train_loss:.4f}, Click AUC: {train_click_auc:.4f}, Order AUC: {train_order_auc:.4f}, Combined: {train_combined:.4f}")
    print(f"Val   - Loss: {val_loss:.4f}, Click AUC: {val_click_auc:.4f}, Order AUC: {val_order_auc:.4f}, Combined: {val_combined:.4f}")
    print(f"LR: {optimizer.param_groups[0]['lr']:.2e}")
    
    # En iyi modeli kaydet
    if val_combined > best_val_score:
        best_val_score = val_combined
        best_model_state = model.state_dict().copy()
        print(f"✅ En iyi model kaydedildi! Score: {best_val_score:.4f}")

# En iyi modeli yükle
if best_model_state is not None:
    model.load_state_dict(best_model_state)
    print(f"\n✅ En iyi model yüklendi. Final validation score: {best_val_score:.4f}")

print("\n🎉 Model eğitimi tamamlandı!")

# Model checkpointing
print("PyTorch Two-Tower modeli kaydediliyor...")
torch.save({
    'model_state_dict': model.state_dict(),
    'model_config': model_config,
    'user_id_map': user_id_map,
    'content_id_map': content_id_map,
    'best_val_score': best_val_score,
    'tokenizer_name': model.bert_model_name
}, 'c:/Users/pc/Desktop/trendyol_hekaton/two_tower_model.pt')

print(f"\n📊 Final PyTorch Two-Tower Model Performance:")
print(f"Architecture: Two-Tower with Turkish BERT")
print(f"BERT Model: dbmdz/bert-base-turkish-cased")
print(f"Final Validation Score: {best_val_score:.6f}")
print(f"Model Parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"Trainable Parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

print("\n🚀 İki kuleli mimari başarıyla oluşturuldu ve eğitildi!")
print("📝 Sorgu kulesi: BERT + Kullanıcı Embedding")
print("🛍️  Ürün kulesi: İçerik Embedding")  
print("🎯 Çıktı başlıkları: Tıklama + Sipariş tahminleri")

# Model inference fonksiyonu tanımla
def load_trained_model(checkpoint_path):
    """Eğitilmiş modeli yükle"""
    checkpoint = torch.load(checkpoint_path, map_location=device)
    
    loaded_model = TwoTowerModel(checkpoint['model_config']).to(device)
    loaded_model.load_state_dict(checkpoint['model_state_dict'])
    loaded_model.eval()
    
    return loaded_model, checkpoint['user_id_map'], checkpoint['content_id_map']

print("\n💾 Model yükleme fonksiyonu tanımlandı: load_trained_model()")

🚀 Two-Tower model eğitimi başlıyor...

EPOCH 1/3
Eğitim başlıyor...
